## SqueezeNet
- Paper : (2016)_Squeezenet-Alexnet level accuracy with 50x fewer parameters and 0.5MB Model size

## Introduction
- More efficient distributed training
- Less overhead when exporting new models to clients
- Feasible FPGA and embedded deployment
- AlexNet과 비슷한 성능을 내며 파라미터는 50배나 적고 모델 사이즈는 0.5MB밖에 안되는 Squeezenet제시 
- **논문의 저자들은 큰 activation map을 가지고 있을수록 성능이 더 높다는 것에서 영감을 얻음**

## Squeezenet 전략(Strategies)
- **(1) Replace 3x3 filters with 1x1 filters**
    - 모든 3x3 convolution filter를 1x1 필터로 교체
    - 1x1 필터가 3x3 필터에 비해 9배나 더 적은 파라미터를 가지고 있기 때문

- **(2) Decrease the number of input channels to 3x3 filters**
    - 하나의 모델 레이어에 전부 3x3 필터로 구성되어있다면 파라미터의 총 수는 (input channel) x (number of filters) x (3x3)개
    - 3x3 필터 자체의 수를 줄이고 이에 더해 3x3으로 들어가는 input channel의 수도 줄임
    - 논문에서는 squeeze layer를 사용하여 input channel -> 3x3 filter로의 수를 줄여버립니다.

- **(3) Downsample late in the network so that convolution layers have large activation**
    - Downsampling part를 네트워크 후반부에 집중사용
    - 보통 downsample은 max(or average) pooling 또는 필터 자체의 stride를 높이는 방식으로 이미지의 spatial resolution을 줄임
    - 이렇게 줄여서 한 번에 필터가 볼 수 있는 영역을 좁히면서 해당 이미지의 정보를 압축시키는 것
    - 더 많은 정보들을 한번에 본다는 것.
    
- 전략 요약
     - **1번과 2번은 CNN 전체 파라미터 수를 줄이면서 정확도를 최대한 보존하는 것에 초점을 맞춤.**
     - **3번의 경우는 파라미터 수가 제한된 상황에서 정확도를 최대화 시키는 방식**
     
<hr>


## Fire module
![squeezenet_firemodule](img/squeezenet_firemodule.jpg)

- Fire module은 2개의 layer(Squeeze layer와 Expansion layer)로 이루어짐
- Squeeze layer가 먼저 나온 뒤 Expansion layer가 따라 나옴.
- Fire module 은 총 3가지 하이퍼 파라미터 존재 (s1x1, e1x1, e3x3)
    - s1x1은 squeeze layer에서 1x1 필터의 총 갯수
    - e1x1은 expand layer에서 1x1 필터의 총 갯수
    - e3x3은 expand layer에서 3x3 필터의 총 갯수
    - s1x1의 값을 e1x1 + e3x3 보다 더 작게 설정
![squeezenet](img/squeezenet.png)

<hr>


## Squeezenet Architecture

![squeezenet_architecture](img/squeezenet_architecture.png)

- 1개의 Conv -> maxpooling -> Fire module x8 -> 최종 Conv -> GAP(Global Average Pooling)
    - max poooling을 fire4, fire8, conv10 이후 적용 -> 3번 전략 취함
    - input/output 각각에 1개의 convolution layer를 사용
    - SqueezeNet은 8개의 fire module을 사용
    - SqueezeNet에서 Fully Connected Layer는 전혀 사용되지 않았음
    - **너무 많은 양의 parameter에 맞추다 보면 overfitting이 발생할 확률도 커지며, SqueezeNet에서는 Global Average Pooling을 사용
    - parameter를 더 만들어 내지 않기 때문에 Fully connected layer에 비해서 overfitting 문제에 좀 더 자유로움

- **Squeeze Layer**
    - 1x1 conv로 Pointwise conv 수행 (channel reduction)
    - 1x1 convolution은 input 데이터의 모든 channel을 하나로 합친 뒤 input channel의 수를 줄여서 다음 layer에 전달하는 역할
    
- **Expansion layer**
    - 1 x 1 convolution이 3 x 3 convolution과 섞인 형태
        - 섞어서 사용함으로써 표현력이 증가하고 파라미터의 숫자도 동시에 줄임
        - **적층을 위한 1x1과 3x3 필터의 output activation이 같기 때문에 3x3 필터 들어가기 전 데이터에 1-pixel zero padding 추가**
    - input 채널의 수를 3 x 3 필터를 이용하여 줄임.

- 기타사항
    - Squeeze와 expand layer 모두 ReLU 적용
    - Fire9 module 이후로 dropout 50% 적용
    - 초기 학습률 0.04로 설정 후 점차 감소시킴

<hr>



## DepthWise vs Pointwise
![depthwise_vs_pointwise](img/depthwise_vs_pointwise.png)


## Global Average Pooling
- **Global average pooling은 이전의 convolution layer로 부터 각각의 channel을 전달 받은 다음 모든 값에 대하여 average를 취해줌**

<hr>

## 참고문헌
- https://gaussian37.github.io/dl-concept-mobilenet-and-squeeznet/
- https://jayhey.github.io/deep%20learning/2018/05/26/SqueezeNet/